In [34]:
import numpy as np
import pandas as pd
import seaborn as sns

In [35]:
path = r"E:\Metro1\artifacts\\"

access_df=pd.read_csv(path+"access_data.csv")
egress_df=pd.read_csv(path+"egress_data.csv")

print(access_df.shape, egress_df.shape)

(1048, 74) (715, 76)


In [36]:
# remove duplicate rows with same id column from dataframe
access_df.drop_duplicates(subset ="ID",inplace = True)
egress_df.drop_duplicates(subset ="ID",inplace = True)

In [37]:
def map_age():
    age_mapping = {
    "18-25 yrs": 21.5,
    '> 60 yrs ':65,
    '36-45yrs ':40, 
    '36-45 yrs ':40, 
    '18-25 yrs ':22,
    '26-35 yrs ':30,
    '18-25yrs ':22,
    '26-35yrs ':30, 
    '46-60 yrs ':54,
      '46-60yrs ':54
    }

# Map the 'AGE' column to average values
    access_df['AGE'] = access_df['AGE'].map(age_mapping)
    egress_df['AGE']=egress_df['AGE'].map(age_mapping)

    return access_df,egress_df

access_df,egress_df=map_age()

In [38]:
# convert 2 in RENT_BICYCLE to 4 and 3 to 1 in access_df dataframe
access_df['RENT_BICYCLE'] = access_df['RENT_BICYCLE'].replace({4: 2, 3: 1})

# create a new column WORK where if O_TYPE or D_TYPE is work then it is 1 else 0
access_df['WORK']=0
access_df.loc[(access_df['O_TYPE']=='Work') | (access_df['D_TYPE']=='Work'),'WORK']=1

access_df['MODE']=0


access_df['DEN']=0
access_df['DEN']=access_df['A_INTERSECTIONS']/access_df['A_DISTANCE_MAP']

access_df=access_df[access_df['DEN']<60]

access_df['UNO']=1

access_df['INDEX']=range(len(access_df))

# output columns ID, RENT_BICYCLE,MALE, ACCESS_SLOPE, A_SLOPE_TYPE, A_DISTANCE_MAP, A_TIME_MAP,O_TYPE,D_TYPE,ACCESS_MODE,AGE_C,N_BICYCLES,CARS,
# TWO_WHEELER, OTHERS, A_INTERSECTINS from access_df dataframe to csv file
access_df[['INDEX','ID', 'RENT_BICYCLE', 'MALE', 'ACCESS_SLOPE', 'A_SLOPE_TYPE', 'A_DISTANCE_MAP', 'A_TIME_MAP','O_TYPE','D_TYPE','ACCESS_MODE',
           'AGE_C','N_BICYCLES','CARS','TWO_WHEELER','OTHERS','A_INTERSECTIONS','WORK','DEN','UNO']].to_csv(path+"access_data_analysis.csv",index=False)

In [39]:
egress_df['RENT_BICYCLE']=egress_df['RENT_BICYCLE'].replace({4: 1, 3: 2})

egress_df['WORK']=0
egress_df.loc[(egress_df['O_TYPE']=='Work') | (egress_df['D_TYPE']=='Work'),'WORK']=1

egress_df['DEN']=0
egress_df['DEN']=egress_df['A_INTERSECTIONS']/egress_df['E_DISTANCE_MAP']

egress_df=egress_df[egress_df['DEN']<60]

egress_df['UNO']=1

egress_df['INDEX']=range(len(access_df),len(access_df)+len(egress_df))
# output columns ID, RENT_BICYCLE,MALE, EGRESS_SLOPE, E_SLOPE_TYPE, E_DISTANCE_MAP, 
# E_TIME_MAP,O_TYPE,D_TYPE,EGRESS_MODE,AGE_C,N_BICYCLES,CARS,
# TWO_WHEELER, OTHERS, A_INTERSECTINS from access_df dataframe to csv file
egress_df[['INDEX','ID', 'RENT_BICYCLE','MALE', 'EGRESS_SLOPE', 'E_SLOPE_TYPE', 'E_DISTANCE_MAP', 
           'E_TIME_MAP','O_TYPE','D_TYPE','EGRESS_MODE','AGE_C','N_BICYCLES','CARS','WORK','DEN','UNO']].to_csv(path+"egress_data_analysis.csv",index=True)

In [40]:
# make the value of RENT_BICYCLE as 1 when it is 2 and 2 when it is 1
egress_df['RENT_BICYCLE']=egress_df['RENT_BICYCLE'].replace({2: 1, 1: 2})

# rename some columns in both dataframes
access_df.rename(columns={'A_INTERSECTIONS':'INTERSECTIONS','A_DISTANCE_MAP':'DISTANCE_MAP','A_TIME_MAP':'TIME_MAP','ACCESS_SLOPE':'SLOPE',
                          'ACCESS_MODE':'MODE','A_SLOPE_TYPE':'SLOPE_TYPE',}
                 ,inplace=True)

egress_df.rename(columns={'A_INTERSECTIONS':'INTERSECTIONS','E_DISTANCE_MAP':'DISTANCE_MAP','E_TIME_MAP':'TIME_MAP','EGRESS_SLOPE':'SLOPE',
                            'EGRESS_MODE':'MODE','E_SLOPE_TYPE':'SLOPE_TYPE',}
                    ,inplace=True)




In [41]:
# df=pd.concat([access_df,egress_df])

In [42]:
df.columns

Index(['INDEX', 'ID', 'RENT_BICYCLE', 'MALE', 'SLOPE', 'SLOPE_TYPE',
       'DISTANCE_MAP', 'TIME_MAP', 'O_TYPE', 'D_TYPE', 'MODE', 'AGE_C',
       'N_BICYCLES', 'CARS', 'WORK', 'DEN', 'UNO', 'POS', 'NEG', 'OLD', 'UP',
       'DOWN', 'FLAT', 'LOW', 'MEDIUM', 'HIGH'],
      dtype='object')

In [44]:
df['INDEX']=df.index+1

df[['INDEX','ID', 'RENT_BICYCLE','MALE', 'SLOPE', 'SLOPE_TYPE', 'DISTANCE_MAP', 
           'TIME_MAP','O_TYPE','D_TYPE','MODE','AGE_C','N_BICYCLES','CARS','WORK','DEN','UNO']]

df=pd.read_csv(path+"data_analysis.csv")

df['UP']=0
df['DOWN']=0
df['FLAT']=0

# if SLOPE is positive then make POS as 1 else 0
df.loc[df['SLOPE']>1,'UP']=1
df.loc[df['SLOPE']<-1,'DOWN']=1
df.loc[(df['SLOPE']>=-1) & (df['SLOPE']<=1),'FLAT']=1


df['OLD']=0

df.loc[df['AGE_C']>2,'OLD']=1

df.to_csv(path+"data_analysis.csv",index=False)

In [45]:
# check the number of rows of df where SLOPE is greater than 2
df[df['SLOPE']<=-2].shape

(99, 26)

In [46]:
df['LOW']=0
df['MEDIUM']=0
df['HIGH']=0

# if DISTANCE_MAP is less than 1 then make LOW as 1
df.loc[df['DISTANCE_MAP']<1,'LOW']=1
# if DISTANCE_MAP is greater than 1 and less than 3 then make MEDIUM 1
df.loc[(df['DISTANCE_MAP']>=1) & (df['DISTANCE_MAP']<3),'MEDIUM']=1
# if DISTANCE_MAP is greater than 3 then make HIGH as 1
df.loc[df['DISTANCE_MAP']>=3,'HIGH']=1

In [47]:
df.to_csv(path+"data_analysis.csv",index=False)